# Sequence to sequence recurrent neural network
Here is a simple NN model that sorts strings of integer numbers.

## Architecture
The architecture is based on classic Sequence to Sequence with Attention [1].
The implemented network features simple Elman recurrent network instead of LSTM in [1].

<img src="seq2seq-model.svg">

The input integer numbers are encoded with binary vectors $x_i \in \{-1,1\}^{N_b}$.
The input sequence $x_i$ is fed to the recurrent encoder nodes $h_i$:
$$ h_i = \sigma(W_h x_i + U_h h_{i-1} + b_h), \quad i = 1,\ldots,N_{seq} $$
here $h_0$ is model parameter.

At each decoding step the encoder states $\{h_i\}$ are weighted and summed to context vector $c_i$:
$$ c_i = \sum_{j=1}^{N_{seq}} \alpha_{ij} h_j, \quad i = 1,\ldots,N_{seq} $$
The context $c_i$ is fed to recurrent decoder node $s_i$. The later takes the previous state $s_{i-1}$ along with last output element $\hat{y}_{i-1}$:
$$ s_i = \sigma(W_s \hat{y}_{i-1} + U_s s_{i-1} + C_s c_i + b_s), \quad i = 1,\ldots,N_{seq} $$
again, the initial state $s_0$ is model parameter, $y_{sos}$ -- constant start-of-sequence marker. The output node $t_i$ takes the context $c_i$, state of decoder $s_i$ and last output element:
$$ t_i = \sigma(U_o s_i + V_o \hat{y}_{i-1} + C_o c_i + b_o), \quad i = 1,\ldots,N_{seq} $$
The real output $t_i$ is rounded to binary $\hat{y}_i$
$$ \hat{y}_i = \mathrm{sign} \, t_i $$

The context weighting function $\alpha_{ij}$ is learned as well. At each decoding state $s_{i-1}$ an attention node $e_{ij}$ is calculated for each encoder state $h_j$:
$$ e_{ij} = v_a^T \sigma(W_a s_{s-1} + U_a h_j + b_a), \quad j = 1,\ldots,N_{seq} $$
Then, the weights are normalized with soft-max:
$$ \alpha_{ij} = \frac{\exp(e_{ij})}{\sum_{k}^{N_{seq}} \exp(e_{ik})}, \quad j = 1,\ldots,N_{seq} $$

## Training
The cross-entropy of real output $t_i$ is chosen for loss function:
$$ J = \sum_{i=1}^{N_{seq}} \sum_{j=1}^{N_b} \log \frac{y_{ij} t_{ij} + 1}{2} $$
where $y_{i} \in \{-1,1\}^{N_b}$ -- binary representation of reverence output

In order to speed-up training, the decoder $s_i$ and output $t_i$ nodes are fed with reference vectors $y_i$
$$ s_i = \sigma(W_s y_{i-1} + U_s s_{i-1} + C_s c_i + b_s) $$
$$ t_i = \sigma(U_o s_i + V_o y_{i-1} + C_o c_i + b_o) $$


[1] Bahdanau, Dzmitry, Kyunghyun Cho, and Yoshua Bengio. "Neural machine translation by jointly learning to align and translate." arXiv preprint arXiv:1409.0473 (2014).